# **LIBs n resources**

In [1]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
from nltk import word_tokenize
from nltk import ngrams
import random
import string
import io
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC,NuSVC
from nltk.classify import ClassifierI




import codecs

#import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
#from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from sklearn.feature_extraction.text import CountVectorizer
import sklearn.feature_extraction.text




from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
#from sklearn.naive_bayes import CategoricalNB

from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC,NuSVC

#from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
#from lightgbm.sklearn import LGBMClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate

#from mlxtend.preprocessing import DenseTransformer
#from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import make_scorer

import time
import statistics


from sklearn.metrics import cohen_kappa_score

In [2]:
data = open("D:\\#Data_science\\Resesrch\\2\\Python\\0 resources\stratified_data_set.txt",encoding='utf-8').read()
data_set = list(data.split('\n'))
#random.shuffle(data_set)
data_set = data_set[3:]
#data_set = data_set[626:]
dataset_size=len(data_set)
print(dataset_size)

sin_diacritics=['ං','ඃ','ා','ැ','ෑ','ි','ී','ු','ූ','ෘ','ෙ','ේ','ෛ','ො','ෝ','ෞ','ෟ','ෲ','ෳ','්','ා']

1938


In [3]:
stem_f = open("stem_dictionary.txt",encoding='utf-8').read()
stemDictManual = dict([l.split('\t') for l in stem_f.split('\n')])
len(stemDictManual)

27195

# **preprocessing (stop+stemming)**

In [3]:
!pip install emoji

In [4]:
from string import *
import re
import emoji


def stopWordRemoval(comment):
    stop_words = open("D:\\#Data_science\\Resesrch\\2\\Python\\0 resources\StopWords.txt",encoding='utf-8').read() 
    sentence = ""
    word_tokens = word_tokenize(comment)
    for w in word_tokens: 
        if w not in stop_words: 
            sentence = sentence + " "+ w
    
    return(sentence)

def rem_diacritics(txt):
    for d in sin_diacritics:
        if d in txt:
            txt=txt.replace(d,'')
    return txt

def get_cleaned_sentence(sentence): #keep only sinhaa characters
    new_sentence = ""
    
    pre_char=''
    chartmp=''
    for char in sentence:
        if pre_char == '.' and char != ' ':# නෙමෙයි.ඒත් :[නෙමෙයි,ඒත්] 
          chartmp=char
          char=' '

        if char not in punctuation :
            if not char.isdigit() :
                if not re.match(r'[a-zA-Z]+', char):
                  if char not in emoji.UNICODE_EMOJI:
                      new_sentence = new_sentence + char

        if chartmp!='' and chartmp not in punctuation :
            if not chartmp.isdigit() :
                if not re.match(r'[a-zA-Z]+', chartmp):
                  if chartmp not in emoji.UNICODE_EMOJI:
                      new_sentence = new_sentence + chartmp
            chartmp=''

        pre_char=char
        
    return new_sentence

#print(get_cleaned_wordList('නෙමෙයි. ඒත් මිනි.සුන් වු අප.ට සිතන්න යමක්'))


def get_word_corpus(dataSet):#all uique words in to a list n sorted
  word_corpus=[]

  for line in dataSet:
    line=line.strip()
    line_set=line.split(' | ')
    
    for word in get_cleaned_sentence(line_set[1]).split(' '):
      if word not in word_corpus and word!='':
        word_corpus.append(word)
  
  word_corpus.sort()

  return word_corpus

#unique words in their frequency in p and n classes are recorded
#then its cheked and recorded whether the word is mostly presented in p or n classes, 
#with a distribution score as ((n-p)^2)/(n+p)
def get_word_corpus_with_distribution(dataSet):
  word_corpus=dict()

  for line in dataSet:
    line=line.strip()
    line_set=line.split(' | ')
    for word in get_cleaned_sentence(line_set[1]).split(' '):
        if word!='':
            if word not in word_corpus.keys() and word!='':
                if line_set[0]=='p':
                    word_corpus[word]=dict()
                    word_corpus[word]['pos']=1
                    word_corpus[word]['neg']=0
                elif line_set[0]=='n':
                    word_corpus[word]=dict()
                    word_corpus[word]['neg']=1
                    word_corpus[word]['pos']=0
            else:
                if line_set[0]=='p':
                    word_corpus[word]['pos']+=1
                elif line_set[0]=='n':
                    word_corpus[word]['neg']+=1

  word_corpus=dict(sorted(word_corpus.items()))

  for key in word_corpus:
    n=word_corpus[key]['neg']
    p=word_corpus[key]['pos']
    
    if n>p:a=n;b=p
    else:a=p;b=n
    
    sum_=a+b
    if sum_==1:sum_+=2
    rat=((a-b)*(a-b))/(sum_)
     
    if n>p:rat=rat*-1
    word_corpus[key]['distribuionScore']=round(rat,2)

  return word_corpus

In [5]:
from collections import defaultdict

In [6]:
def corpus_based_stemmer(dataSet):#stems are present as words in sorted word corpus
  wordCorpus=get_word_corpus(dataSet)
    
  stemmed_corpus_1=defaultdict(list)

  for stem in wordCorpus:
        for w in wordCorpus:
            if w.startswith(stem) and stem!=w:
                stemmed_corpus_1[stem].append(w) 
  
  for k,v in stemmed_corpus_1.items():
    stemmed_corpus_1[k]=[i for i in v if len((i))<=len((k))+2]#to not to stem longer words into shorter words 
    print(k,stemmed_corpus_1[k])
    
  stemmed_corpus_2=dict()
  for k in sorted(stemmed_corpus_1.keys(), key=len, reverse=True):
    stemmed_corpus_2[k]=stemmed_corpus_1[k]
    
  stemmed_corpus=dict()
  for k,v in stemmed_corpus_2.items():
    for i in v:
        if i not in stemmed_corpus.keys():
            stemmed_corpus[i]=k

  return stemmed_corpus

def corpus_based_stemmer_w_suffix(dataSet):#stems are present as words in sorted word corpus
  wordCorpus=get_word_corpus(dataSet)
  stemmed_corpus=dict()
  suffix_file = open("D:\\#Data_science\\Resesrch\\2\\Python\\0 resources\suffixes.txt",encoding='utf-8').read()
  suffix_lst = suffix_file.split()
    
  for stem in wordCorpus:
        for w in wordCorpus:
            if w.startswith(stem) and stem!=w:
                for suf in suffix_lst:
                    if stem+suf==w:
                        stemmed_corpus[w]=stem

  return stemmed_corpus

In [7]:

#earliast suffix in the list removed
def suffix_removal_stemmer_1(dataSet):
  wordCorpus=get_word_corpus(dataSet)
  stemmed_corpus=dict()
  suffix_file = open("D:\\#Data_science\\Resesrch\\2\\Python\\0 resources\suffixes.txt",encoding='utf-8').read()
  suffix_lst = suffix_file.split()
  #print(len(suffix_lst))

  c=0
  for w in wordCorpus[1:]:
    
    #suffixes
    x=0 
    for suf in suffix_lst:
      if w.endswith(suf):
        c+=1
        stem=w[:len(w)-len(suf)]
        stemmed_corpus[w]=stem
        break

  print(len(wordCorpus))
  print(c)

  return stemmed_corpus


def find_longest_suffix(lst):
    list_len = [len(i) for i in lst]
    return (min(list_len))
def suffix_removal_stemmer_2(dataSet):
  wordCorpus=get_word_corpus(dataSet)
  stemmed_corpus=dict()
  suffix_file = open("D:\\#Data_science\\Resesrch\\2\\Python\\0 resources\suffixes.txt",encoding='utf-8').read()
  suffix_lst = suffix_file.split()
  #print(len(suffix_lst))

  c=0
  for w in wordCorpus[1:]:
    w_len=len(w)
    matched_sufs=[]
    for suf in suffix_lst:
      if w.endswith(suf):
        matched_sufs.append(suf)
    if len(matched_sufs)==1:
      c+=1
      stem=w[:w_len-len(matched_sufs[0])]
      stemmed_corpus[w]=stem
    elif len(matched_sufs)>1:
      #print(0)
      c+=1
      longest_suf=find_longest_suffix(matched_sufs)
      stem=w[:w_len-longest_suf]
      stemmed_corpus[w]=stem
    elif len(matched_sufs)==0:
        stemmed_corpus[w]=w
        
  # print(len(wordCorpus))
  # print(c)

  return stemmed_corpus


In [8]:
# def get_ngrams(w,n):
#     return list(w[i:i+n] for i in range( 0,len(w)-(n-1) ))

def get_ngrams(w,n_grams):
    ngrams=[]
    for ng in n_grams:
        ngrams=ngrams+list(w[i:i+ng] for i in range( 0,len(w)-(ng-1) ))
    return ngrams

def char_ngram_stemmer(dataSet):
    n=[3,4] #ngram used
    cutoff_val=0.73
    
    #wordCorpus=get_word_corpus(dataSet)
    wordCorpus_wt_Dist=get_word_corpus_with_distribution(dataSet)
    wordCorpus=list(wordCorpus_wt_Dist.keys()) 
    wordCorpus.sort(key=len)# sorted by length first then same length words are sorted alphabeticaly
    wordCorpus=[i for i in wordCorpus[::-1]] #inverse the list : longer words stemmed firs. avoids too much stemming and increases acc
    
    stemmed_corpus=dict()   
    ngram_corpus=dict()
    for word in wordCorpus:
        ngram_corpus[word]=get_ngrams(word,n)
    
#    sim_score_distribution=dict()
    tmp=dict()#get all similarity_score>=cutoff_val words for a particular word in wordCorpus 
    for key1 in wordCorpus:
        tmp[key1]=[]
        lst1=ngram_corpus[key1]
        if len(lst1)==0:continue
            
        for key2 in wordCorpus[wordCorpus.index(key1)+1:]:
            if key2!=key1:
                lst2=ngram_corpus[key2]
                if len(lst2)==0:continue
                
                #method 1
                common_ngrams=len(set(lst1).intersection(lst2))
                unique_ngrams=(len(lst1)-common_ngrams)+(len(lst2)-common_ngrams)
                similarity_score=2*common_ngrams/unique_ngrams

                
                if similarity_score>=cutoff_val:
                    tmp[key1].append(key2)

    for key in tmp:
        pos=[]
        neg=[]
        neu=[]
        dist_score=wordCorpus_wt_Dist[key]['distribuionScore']
        if dist_score>=0.5:pos.append((key,dist_score))
        elif dist_score<0.5 and dist_score>-0.5 :neu.append((key,dist_score))
        elif dist_score<=-0.5:neg.append((key,dist_score))
            
        for i in tmp[key]:
            dist_score=wordCorpus_wt_Dist[key]['distribuionScore']
            if dist_score>=0.5:pos.append((i,dist_score))
            elif dist_score<0.5 and dist_score>-0.5 :neu.append((i,dist_score))
            elif dist_score<=-0.5:neg.append((i,dist_score))
        
        for i in pos:
            if i[0] not in stemmed_corpus.keys():
                stemmed_corpus[i[0]]=key+'s1'
        for i in neg:
            if i[0] not in stemmed_corpus.keys():
                stemmed_corpus[i[0]]=key+'s2'
        for i in neu:
            if i[0] not in stemmed_corpus.keys():
                stemmed_corpus[i[0]]=key+'s3'


    print('len stemmed_corpus',len(stemmed_corpus.keys()))
    
    return stemmed_corpus

In [14]:
from statistics import mean
def successor_variety_stemmer_cutoff(dataSet):
  wordCorpus=get_word_corpus(dataSet)
  stemmed_corpus=dict()

  sv_wordCorpus=dict()
  for word in wordCorpus:
    wordLen=len(word)
    substring=''

    word_sv_data=[]
    for let in word:
      substring=substring+let
      substring_sv_val=0 #how many successor letters in words starting with same substring
      substring_sv_words=[] #store coresponding words of sv vals
      
      if let not in []:#['ං','ඃ','ා','ැ','ෑ','ි','ී','ු','ූ','ෘ','ෙ','ේ','ෛ','ො','ෝ','ෞ','ෟ','ෲ','ෳ','්','ා']:  
        for w in wordCorpus:
          if w!=word and w.startswith(substring):
            substring_sv_val+=1
            substring_sv_words.append(w)
        
        if substring_sv_val!=0:
          substring_sv_data={
              'substring':substring,
              'substring_sv_val':substring_sv_val,
              'substring_sv_words':substring_sv_words,
          }
          word_sv_data.append(substring_sv_data)

    sv_wordCorpus[word]=word_sv_data

  print(len(list(sv_wordCorpus.keys())))
  cutOffVal=2
  for key in sv_wordCorpus:
    for substring_data in sv_wordCorpus[key]:
      if substring_data['substring_sv_val']<=cutOffVal:
        for sv_word in substring_data['substring_sv_words']:
          if sv_word not in stemmed_corpus.keys():
            stemmed_corpus[sv_word]=substring_data['substring']


  return stemmed_corpus


import random
import pickle
sv_WordCorpus=pickle.load(open('maxEnt_sv_wordCorpus','rb'))
sv_WordCorpus_v2=pickle.load(open('maxEnt_sv_wordCorpus_v2','rb'))


def successor_variety_stemmer_max_e(sv_wordCorpus):#0maxEnt_sv_wordCorpus
  stemmed_corpus=dict()
  for word_sv_data in sv_wordCorpus:
    for substring_sv_data in word_sv_data[::-1]:
      ME_Val=substring_sv_data['substring_sv_val']
      # print(ME_Val)
      if ME_Val<=10 and ME_Val>=0.0:
            if len(substring_sv_data['substring_sv_words'])>=0 and len(rem_diacritics(substring_sv_data['substring']))>2:
                # print(substring_sv_data['substring'],substring_sv_data['substring_sv_words'])
                for word in substring_sv_data['substring_sv_words']:
                  if word not in stemmed_corpus.keys():
                    stemmed_corpus[word]=substring_sv_data['substring']
  
  print('len stemmed_corpus',len(stemmed_corpus.keys()))
  return stemmed_corpus

In [92]:
me=defaultdict(int)
def successor_variety_stemmer_max_e(sv_wordCorpus):#0maxEnt_sv_wordCorpus
  stemmed_corpus=dict()
  for word_sv_data in sv_wordCorpus:
    for substring_sv_data in word_sv_data[::-1]:
      ME_Val=substring_sv_data['substring_sv_val']
      me[round(ME_Val,1)]+=1
      if ME_Val<=2 and ME_Val>=0.0:
            if len(substring_sv_data['substring_sv_words'])>=0 and len(rem_diacritics(substring_sv_data['substring']))>3:
                # print(substring_sv_data['substring'],substring_sv_data['substring_sv_words'])
                for word in substring_sv_data['substring_sv_words']:
                  if word not in stemmed_corpus.keys():
                    stemmed_corpus[word]=substring_sv_data['substring']

  for i,j in (me.items()):print(i,j)
  print('len stemmed_corpus',len(stemmed_corpus.keys()))
  return stemmed_corpus

## compare stemmers  

In [93]:
#without stemming 75.28

#stemsDict=corpus_based_stemmer(data_set) #1299 words #74.87% #length clusterd #75.28
# stemsDict=corpus_based_stemmer_w_suffix(data_set) #75.23

#stemsDict=suffix_removal_stemmer_1(data_set)#5965 #73.01%
#stemsDict=suffix_removal_stemmer_2(data_set)#3136 #longest stem LR 72.34% | shortest stem: MNB 74.05%

#stemsDict=successor_variety_stemmer_cutoff(data_set)#MNB without diacritics 74.56% | with diacritrics 74.45% 
stemsDict=successor_variety_stemmer_max_e(sv_WordCorpus_v2) #75.33% mnb


#stemsDict=char_ngram_stemmer(data_set) #81%

#stemsDict=stemDictManual #674 #74.nn%

0.1 223
0.5 2332
0.8 591
0.0 17551
1.0 1318
0.4 655
5.2 76
12.2 272
0.2 356
0.6 465
1.4 429
0.9 614
1.8 285
0.3 519
1.9 385
1.6 706
0.7 445
2.2 271
4.6 87
2.3 176
3.9 154
1.3 432
5.3 290
3.7 230
1.2 517
4.0 110
11.0 35
13.0 6
3.1 201
12.3 169
2.0 382
1.1 904
4.2 121
13.4 294
2.6 354
2.1 449
7.7 112
1.5 526
6.6 113
11.8 420
6.3 137
1.7 189
2.8 158
3.5 189
5.1 59
7.0 374
4.9 121
9.4 131
6.0 361
2.7 163
4.4 126
9.5 251
3.2 305
2.4 204
2.9 189
11.1 114
4.3 118
4.7 124
8.9 30
4.1 126
3.6 116
5.6 28
4.8 140
3.3 171
5.0 162
5.9 72
10.8 356
6.4 21
3.4 94
3.8 68
2.5 185
6.9 28
12.1 35
5.4 185
9.2 17
7.8 18
10.7 45
7.4 67
8.4 240
6.8 19
7.3 17
6.1 58
7.1 25
7.2 19
6.2 37
7.5 36
8.3 47
3.0 132
5.8 45
9.3 73
5.5 11
6.7 13
4.5 100
6.5 48
11.7 49
8.1 5
10.5 3
8.8 12
5.7 51
8.2 24
10.6 3
11.4 5
8.5 1
8.7 1
13.2 1
12.8 1
len stemmed_corpus 1388


In [94]:
ensembler('\nLogisticRegression',LogisticRegression())
ensembler('\nMultinomialNB',MultinomialNB())                      
ensembler('\nComplementNB',ComplementNB()) 


LogisticRegression
conf_matrix :  [698, 271, 252, 717]
accuracy :  73.01341589267287
fscore :  0.7327542156361777
cohen_kappa_score:  0.4602683178534571



MultinomialNB
conf_matrix :  [749, 220, 281, 688]
accuracy :  74.14860681114551
fscore :  0.7330847096430474
cohen_kappa_score:  0.48297213622291024



ComplementNB
conf_matrix :  [754, 215, 300, 669]
accuracy :  73.42621259029927
fscore :  0.722072315164598
cohen_kappa_score:  0.46852425180598556




# **train classifiers**

In [16]:
def prepareData(Dataset1,n):
    
  actual=[]
  Dataset2=[]#'np | ' removed

  t1_start=0
  t_prepareData=0
  
  for linex in Dataset1:
      if 'p | ' in linex or 'n | ' in linex :
          line_split=linex.split(' | ')
          line=line_split[1]
          line = line.strip()
        
          line=stopWordRemoval(line)
          line=get_cleaned_sentence(line)
          
#           t1_start=time.time()
          line=get_stemmed_sentence(line,stemsDict)
#           t_prepareData+=time.time()-t1_start
          #line=RemDiacritics(line)  
          Dataset2.append(line)
          
          if line_split[0]=='p':
              actual.append(1)
          elif line_split[0]=='n':
              actual.append(0)  


  t2_start=time.time()
  count_vect_single_uni=CountVectorizer(analyzer='word',ngram_range=(1,1))# char_wb, char, word, (3,4) , (1,1),(n[0],n[1])
  data_single_uni=count_vect_single_uni.fit_transform(Dataset2)
  t_prepareData=t_prepareData+(time.time()-t2_start)

  return (actual,data_single_uni,t_prepareData)
    
   
def get_stemmed_sentence(sentence,stemmed_dict):
  for word in sentence.split(' '):
    if word in stemmed_dict.keys():
      sentence=sentence.replace(word,stemmed_dict[word])
  return sentence
    
    
    
def RemDiacritics(comment):
    tmp_str=''
    for word in comment.split(' '):
        tmp_word=''
        L=int((len(word)*0.75)-((len(word)*0.75)%1) )          
        for char in word[:L]: 
            if char not in sin_diacritics:
                tmp_word=tmp_word+char
        tmp_str=tmp_str+tmp_word+' '
    return tmp_str

# **sentiment analyzer**

## cross validation

In [17]:
def calculate_Fscore(conf_matrix):# order : 0 tn, 1 fp, 2 fn, 3 tp      
    accuracy=(conf_matrix[3]+conf_matrix[0])/(conf_matrix[3]+conf_matrix[2]+conf_matrix[0]+conf_matrix[1])*100
    precision=conf_matrix[3]/(conf_matrix[3]+conf_matrix[1])
    recall=conf_matrix[3]/(conf_matrix[3]+conf_matrix[2])
    speficity=conf_matrix[0]/(conf_matrix[0]+conf_matrix[1])
    fscore=2*(precision*recall)/(precision+recall)
    NegPredictiveVal=conf_matrix[0]/(conf_matrix[0]+conf_matrix[2])
    print('conf_matrix : ',conf_matrix)  
    print('accuracy : ',accuracy) 
    print('fscore : ',fscore)
#     print('precision : ',precision)
#     print('recall : ',recall)
#     print('NegPredictiveVal : ',NegPredictiveVal)
#     print('speficity : ',speficity)

    #print(log_loss)
    return([round(accuracy,3),round(fscore,3)])

In [18]:
multi_score={
    'accuracy' : make_scorer(accuracy_score),
    'f1score' : make_scorer(f1_score),
    'precision': make_scorer(precision_score),
    'auc': make_scorer(roc_auc_score)
          }

def ensembler(c_name,EClassifier):
  print(c_name)

  e_actual,e_data_single_uni,t_prepData=prepareData(data_set,0)
    
  #for syllables testing use:
  #e_actual,e_data_single_uni,t_prepData=prepareSyllablesData(data_set,0)
    
    
  kfold_val=6
  predictions=cross_val_predict(EClassifier,e_data_single_uni,e_actual,cv=kfold_val)
  cm=list(confusion_matrix(e_actual,predictions).ravel())
  calculate_Fscore(cm)

  print('cohen_kappa_score: ',cohen_kappa_score(e_actual,predictions))
    
  scores=cross_validate(EClassifier,e_data_single_uni,e_actual,cv=kfold_val)
  t_testData=scores['score_time'].mean()
  t_Total=t_testData+t_prepData
#   print('time: ',t_Total)
  print('\n')


# TEST FEW N VALUES
# def ensembler(c_name,EClassifier):
#     print(c_name)
#     lst=[]
#     for i in [[(2, 2), (3, 3), (4, 4), (5, 5), (6, 6)]]:
#         #for graphs
# #     [[(2, 2), (3, 3), (4, 4), (5, 5), (6, 6)],[(2,2),(2,3),(2,4),(2,5),(2,6),(2,7)],[(3,3),(3,4),(3,5),(3,6),(3,7)],[(4,4),(4,5),(4,6),(4,7)]]:
#         dct=dict()
#         for n in i:
#           print(str(n))

#           e_actual,e_data_single_uni,t_prepData=prepareData(data_set,n)

#           kfold_val=6
#           predictions=cross_val_predict(EClassifier,e_data_single_uni,e_actual,cv=kfold_val)
#           cm=list(confusion_matrix(e_actual,predictions).ravel())
#           print('cohen_kappa_score: ',cohen_kappa_score(e_actual,predictions))
# #           dct[n]=calculate_Fscore(cm)
# #           print('\n')
#         lst.append(dct)
# #     print(lst)
#     for l in lst:print(l)
    
# #GET AVG TIME   
# def ensembler(c_name,EClassifier):
#     print(c_name)
#     tmpLst=[]
#     for i in range(0,5):
#       e_actual,e_data_single_uni,t_prepData=prepareData(data_set,1)

#       kfold_val=6

#       scores=cross_validate(EClassifier,e_data_single_uni,e_actual,cv=kfold_val)
#       t_testData=scores['score_time'].mean()
#       t_Total=t_testData+t_prepData
#       tmpLst.append(t_Total)
#     #      print('time: ',t_Total)

#     print('time: ',statistics.mean(tmpLst))
#     print('\n')


In [146]:
#print('total unique words',len(get_word_corpus(data_set)))

ensembler('\nLogisticRegression',LogisticRegression())
#print(len(stemsDict))
ensembler('\nMultinomialNB',MultinomialNB())                      
ensembler('\nComplementNB',ComplementNB())   

# successor_variety_stemmer
# for i in range (0,50):
#     stemsDict=successor_variety_stemmer(sv_WordCorpus)
#     ensembler('\nLogisticRegression',LogisticRegression())

                       
 
#time:  0.7891678810119629
#time:  0.09294867515563965


LogisticRegression
conf_matrix :  [721, 248, 239, 730]
accuracy :  74.87100103199175
fscore :  0.7498715973292245
cohen_kappa_score:  0.4974200206398349



MultinomialNB
conf_matrix :  [757, 212, 284, 685]
accuracy :  74.40660474716202
fscore :  0.7341907824222937
cohen_kappa_score:  0.4881320949432405



ComplementNB
conf_matrix :  [767, 202, 315, 654]
accuracy :  73.32301341589267
fscore :  0.7167123287671233
cohen_kappa_score:  0.4664602683178535




In [19]:
#import nltk
#nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Pradeep
[nltk_data]     Jayasuriya\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [89]:
ensembler('\nLogisticRegression',LogisticRegression())
ensembler('\nSVC',SVC())
ensembler('\nLinearSVC',LinearSVC())                       
ensembler('\nNuSVC',NuSVC())                       

#ensembler('\nGaussianNB',GaussianNB())                     
ensembler('\nBernoulliNB',BernoulliNB())                        
ensembler('\nMultinomialNB',MultinomialNB())                      
ensembler('\nComplementNB',ComplementNB())                      
#ensembler('\nCategoricalNB',CategoricalNB()) 

rfClassifier=RandomForestClassifier(n_estimators= 800, min_samples_split= 10, min_samples_leaf= 4, max_features='sqrt', max_depth=20, bootstrap=True)
ensembler('\nRandomForestClassifier',rfClassifier)  

#poor results close to random selection 
knnClassifier=KNeighborsClassifier(weights= 'distance', n_neighbors= 10, n_jobs= -1, leaf_size= 1, algorithm='kd_tree'  )                 
ensembler('\nKNeighborsClassifier',knnClassifier)  

ensembler('\nDecisionTreeClassifier',DecisionTreeClassifier())                       
ensembler('\nAdaBoostClassifier',AdaBoostClassifier())
#ensembler('\nXGBClassifier',XGBClassifier())                                            
ensembler('\nGradientBoostingClassifier',GradientBoostingClassifier())                            
#ensembler('\nLGBMClassifier',LGBMClassifier())


LogisticRegression
conf_matrix :  [707, 262, 249, 720]
accuracy :  73.63261093911248
fscore :  0.7380830343413634
cohen_kappa_score:  0.4726522187822497



SVC
conf_matrix :  [711, 258, 295, 674]
accuracy :  71.46542827657379
fscore :  0.7091004734350341
cohen_kappa_score:  0.4293085655314758



LinearSVC
conf_matrix :  [683, 286, 239, 730]
accuracy :  72.91021671826626
fscore :  0.7355163727959697
cohen_kappa_score:  0.45820433436532504



NuSVC
conf_matrix :  [719, 250, 283, 686]
accuracy :  72.49742002063984
fscore :  0.7202099737532808
cohen_kappa_score:  0.4499484004127967



BernoulliNB
conf_matrix :  [551, 418, 177, 792]
accuracy :  69.2982456140351
fscore :  0.7269389628269849
cohen_kappa_score:  0.38596491228070173



MultinomialNB
conf_matrix :  [752, 217, 277, 692]
accuracy :  74.50980392156863
fscore :  0.7369542066027688
cohen_kappa_score:  0.4901960784313726



ComplementNB
conf_matrix :  [758, 211, 313, 656]
accuracy :  72.96181630546955
fscore :  0.7145969498910675
coh

C:\Users\Pradeep Jayasuriya\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_base.py:462: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
C:\Users\Pradeep Jayasuriya\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_base.py:462: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
C:\Users\Pradeep Jayasuriya\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_base.py:462: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
C:\Users\Pradeep Jayasuriya\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_base.py:462: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
C:\Users\Pradeep Jayasuriya\AppData\Local\Programs\P

conf_matrix :  [360, 609, 135, 834]
accuracy :  61.60990712074303
fscore :  0.691542288557214
cohen_kappa_score:  0.2321981424148607



DecisionTreeClassifier


C:\Users\Pradeep Jayasuriya\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_base.py:462: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
C:\Users\Pradeep Jayasuriya\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_base.py:462: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
C:\Users\Pradeep Jayasuriya\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_base.py:462: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
C:\Users\Pradeep Jayasuriya\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\neighbors\_base.py:462: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


conf_matrix :  [613, 356, 276, 693]
accuracy :  67.3890608875129
fscore :  0.686818632309217
cohen_kappa_score:  0.34778121775025794



AdaBoostClassifier
conf_matrix :  [591, 378, 300, 669]
accuracy :  65.01547987616098
fscore :  0.6636904761904763
cohen_kappa_score:  0.30030959752321984



GradientBoostingClassifier
conf_matrix :  [734, 235, 345, 624]
accuracy :  70.07223942208462
fscore :  0.6827133479212253
cohen_kappa_score:  0.4014447884416925




#### char vs char_wb

In [ ]:
Data=['doing how12 are34']
count_vect_single_uni=CountVectorizer(analyzer='char',ngram_range=(5,5))
data_single_uni=count_vect_single_uni.fit_transform(Data)
print(count_vect_single_uni.vocabulary_)

Data2=['doing how12 are34']
count_vect_single_uni2=CountVectorizer(analyzer='char_wb',ngram_range=(5,5))
data_single_uni2=count_vect_single_uni2.fit_transform(Data2)
print(count_vect_single_uni2.vocabulary_)

# RESULTS

### time

In [ ]:
word no stem

LogisticRegression
time:  0.06444430351257324

SVC
time:  0.14610473314921063

LinearSVC
time:  0.06746137142181396

NuSVC
time:  0.13309931755065918

BernoulliNB
time:  0.062645157178243

MultinomialNB
time:  0.060965538024902344

ComplementNB
time:  0.06496540705362956

RandomForestClassifier
time:  0.1839064359664917

KNeighborsClassifier
time:  0.1807210048039754

DecisionTreeClassifier
time:  0.07012466589609782

AdaBoostClassifier
time:  0.08545633157094319

GradientBoostingClassifier
time:  0.06710004806518555

In [ ]:
word with stem

LogisticRegression
time:  0.13241986433664957

SVC
time:  0.19811793168385822

LinearSVC 
time:  0.13984882831573486

NuSVC
time:  0.19833660125732422

BernoulliNB
time:  0.13632190227508545

MultinomialNB
time:  0.12107054392496745

ComplementNB
time:  0.1312838395436605

RandomForestClassifier
time:  0.2467422882715861

KNeighborsClassifier
time:  0.24372013409932453

DecisionTreeClassifier
time:  0.12196969985961914

AdaBoostClassifier
time:  0.13451906045277914

GradientBoostingClassifier
time:  0.1311038335164388

In [ ]:
char

LogisticRegression
time:  0.34863317012786865

SVC
time:  0.5810085137685139

LinearSVC
time:  0.3692919810612996

NuSVC
time:  0.44141479333241784

BernoulliNB
time:  0.2788412570953369

MultinomialNB
time:  0.3812855879465739

ComplementNB
time:  0.27401479085286456

RandomForestClassifier
time:  0.4951939185460409

KNeighborsClassifier
time:  0.40275247891743976

DecisionTreeClassifier
time:  0.28117191791534424

AdaBoostClassifier
time:  0.3036406437555949

GradientBoostingClassifier
time:  0.29036064942677814

### time averaged

In [ ]:
word no stemming


LogisticRegression
time:  0.03841692606608073

SVC
time:  0.0905786911646525

LinearSVC
time:  0.055505347251892094

NuSVC
time:  0.06707580884297688

BernoulliNB
time:  0.036753908793131514

MultinomialNB
time:  0.04060922463734945

ComplementNB
time:  0.03664030234018962

RandomForestClassifier
time:  0.15048072338104249

KNeighborsClassifier
time:  0.15047696431477864

DecisionTreeClassifier
time:  0.03622949918111165

AdaBoostClassifier
time:  0.05400792757670085

GradientBoostingClassifier
time:  0.03808191617329915


In [ ]:
word stemmed

LogisticRegression
time:  0.11951778729756672

SVC
time:  0.15680286089579265

LinearSVC
time:  0.0931069135665893

NuSVC
time:  0.13259034951527912

BernoulliNB
time:  0.09301144282023112

MultinomialNB
time:  0.092702317237854

ComplementNB
time:  0.0853879451751709

RandomForestClassifier
time:  0.20433928966522216

KNeighborsClassifier
time:  0.21553811232248943

DecisionTreeClassifier
time:  0.09712134202321371

AdaBoostClassifier
time:  0.11907191276550293

GradientBoostingClassifier
time:  0.12143030166625976

In [ ]:
char no stem


LogisticRegression
time:  0.31989802519480387

SVC
time:  0.6044735034306844

LinearSVC
time:  0.31010434627532957

NuSVC
time:  0.48147594928741455

BernoulliNB
time:  0.31935735543568927

MultinomialNB
time:  0.3137272198994954

ComplementNB
time:  0.30649627049764

RandomForestClassifier
time:  0.5248480161031087

DecisionTreeClassifier
time:  0.36340487003326416

AdaBoostClassifier
time:  0.3667421817779541

GradientBoostingClassifier
time:  0.3238156239191691

In [ ]:
word no stem


LogisticRegression
conf_matrix :  [717, 252, 240, 729]
accuracy :  74.61300309597523
fscore :  0.7476923076923078
precision :  0.7431192660550459
recall :  0.7523219814241486
NegPredictiveVal :  0.7492163009404389
speficity :  0.739938080495356
time:  0.022549986839294434



SVC
conf_matrix :  [740, 229, 305, 664]
accuracy :  72.44582043343654
fscore :  0.7132116004296456
precision :  0.7435610302351624
recall :  0.6852425180598555
NegPredictiveVal :  0.7081339712918661
speficity :  0.7636738906088751
time:  0.06392192840576172



LinearSVC
conf_matrix :  [677, 292, 248, 721]
accuracy :  72.13622291021672
fscore :  0.7275479313824419
precision :  0.7117472852912142
recall :  0.7440660474716202
NegPredictiveVal :  0.7318918918918919
speficity :  0.6986584107327142
time:  0.021874189376831055



NuSVC
conf_matrix :  [746, 223, 301, 668]
accuracy :  72.96181630546955
fscore :  0.7182795698924732
precision :  0.7497194163860831
recall :  0.6893704850361198
NegPredictiveVal :  0.7125119388729704
speficity :  0.7698658410732714
time:  0.06175490220387777



BernoulliNB
conf_matrix :  [529, 440, 171, 798]
accuracy :  68.47265221878224
fscore :  0.7231536021748981
precision :  0.6445880452342488
recall :  0.8235294117647058
NegPredictiveVal :  0.7557142857142857
speficity :  0.5459236326109391
time:  0.022678573926289875



MultinomialNB
conf_matrix :  [747, 222, 269, 700]
accuracy :  74.66460268317854
fscore :  0.74034902168165
precision :  0.7592190889370932
recall :  0.7223942208462333
NegPredictiveVal :  0.735236220472441
speficity :  0.7708978328173375
time:  0.021281639734903973



ComplementNB
conf_matrix :  [752, 217, 285, 684]
accuracy :  74.09700722394221
fscore :  0.7315508021390374
precision :  0.7591564927857936
recall :  0.7058823529411765
NegPredictiveVal :  0.7251687560270009
speficity :  0.7760577915376677
time:  0.02259806791941325



RandomForestClassifier
conf_matrix :  [716, 253, 285, 684]
accuracy :  72.23942208462333
fscore :  0.7177334732423926
precision :  0.7299893276414088
recall :  0.7058823529411765
NegPredictiveVal :  0.7152847152847153
speficity :  0.7389060887512899
time:  0.11187080542246501



KNeighborsClassifier
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:414: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:414: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:414: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:414: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:414: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
/usr/local/lib/python3.6/dist-packages/sklearn/neighbors/_base.py:414: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "
conf_matrix :  [357, 612, 165, 804]
accuracy :  59.907120743034056
fscore :  0.6742138364779874
precision :  0.5677966101694916
recall :  0.8297213622291022
NegPredictiveVal :  0.6839080459770115
speficity :  0.3684210526315789
time:  0.1360615889231364



DecisionTreeClassifier
conf_matrix :  [619, 350, 271, 698]
accuracy :  67.95665634674923
fscore :  0.692117005453644
precision :  0.666030534351145
recall :  0.7203302373581011
NegPredictiveVal :  0.6955056179775281
speficity :  0.6388028895768834
time:  0.02352861563364665



AdaBoostClassifier
conf_matrix :  [497, 472, 194, 775]
accuracy :  65.63467492260062
fscore :  0.6994584837545126
precision :  0.6214915797914996
recall :  0.7997936016511867
NegPredictiveVal :  0.7192474674384949
speficity :  0.5128998968008256
time:  0.03509970506032308



XGBClassifier
conf_matrix :  [740, 229, 347, 622]
accuracy :  70.27863777089783
fscore :  0.6835164835164835
precision :  0.7309048178613397
recall :  0.6418988648090815
NegPredictiveVal :  0.6807727690892365
speficity :  0.7636738906088751
time:  0.026115059852600098



GradientBoostingClassifier
conf_matrix :  [727, 242, 339, 630]
accuracy :  70.02063983488132
fscore :  0.6844106463878328
precision :  0.7224770642201835
recall :  0.6501547987616099
NegPredictiveVal :  0.6819887429643527
speficity :  0.7502579979360166
time:  0.022968371709187824


In [ ]:
word stemmed


LogisticRegression
conf_matrix :  [779, 190, 213, 756]
accuracy :  79.20536635706914
fscore :  0.789556135770235
precision :  0.7991543340380549
recall :  0.7801857585139319
NegPredictiveVal :  0.7852822580645161
speficity :  0.803921568627451
time:  0.07368632157643636



SVC
conf_matrix :  [809, 160, 282, 687]
accuracy :  77.19298245614034
fscore :  0.7566079295154186
precision :  0.8110979929161747
recall :  0.7089783281733746
NegPredictiveVal :  0.7415215398716773
speficity :  0.8348813209494325
time:  0.12278886636098227



LinearSVC
conf_matrix :  [727, 242, 216, 753]
accuracy :  76.36738906088752
fscore :  0.7668024439918534
precision :  0.75678391959799
recall :  0.7770897832817337
NegPredictiveVal :  0.7709437963944857
speficity :  0.7502579979360166
time:  0.07453091939290364



NuSVC
conf_matrix :  [803, 166, 264, 705]
accuracy :  77.81217750257998
fscore :  0.766304347826087
precision :  0.8094144661308841
recall :  0.7275541795665634
NegPredictiveVal :  0.7525773195876289
speficity :  0.8286893704850361
time:  0.12172063191731772



BernoulliNB
conf_matrix :  [610, 359, 181, 788]
accuracy :  72.13622291021672
fscore :  0.7448015122873346
precision :  0.6870095902353966
recall :  0.8132094943240454
NegPredictiveVal :  0.7711757269279393
speficity :  0.6295149638802889
time:  0.07538366317749023



MultinomialNB
conf_matrix :  [805, 164, 253, 716]
accuracy :  78.48297213622291
fscore :  0.7744726879394267
precision :  0.8136363636363636
recall :  0.7389060887512899
NegPredictiveVal :  0.7608695652173914
speficity :  0.8307533539731682
time:  0.07326225439707439



ComplementNB
conf_matrix :  [805, 164, 254, 715]
accuracy :  78.43137254901961
fscore :  0.7738095238095238
precision :  0.8134243458475541
recall :  0.737874097007224
NegPredictiveVal :  0.7601510859301227
speficity :  0.8307533539731682
time:  0.07409083843231201



RandomForestClassifier
conf_matrix :  [806, 163, 274, 695]
accuracy :  77.45098039215686
fscore :  0.7608100711548988
precision :  0.8100233100233101
recall :  0.717234262125903
NegPredictiveVal :  0.7462962962962963
speficity :  0.8317853457172343
time:  0.1667555570602417



KNeighborsClassifier
conf_matrix :  [714, 255, 314, 655]
accuracy :  70.63983488132095
fscore :  0.6971793507184672
precision :  0.7197802197802198
recall :  0.675954592363261
NegPredictiveVal :  0.6945525291828794
speficity :  0.7368421052631579
time:  0.18573105335235596



DecisionTreeClassifier
conf_matrix :  [707, 262, 289, 680]
accuracy :  71.56862745098039
fscore :  0.7116692830978545
precision :  0.721868365180467
recall :  0.7017543859649122
NegPredictiveVal :  0.7098393574297188
speficity :  0.7296181630546955
time:  0.07427871227264404



AdaBoostClassifier
conf_matrix :  [756, 213, 276, 693]
accuracy :  74.76780185758514
fscore :  0.7392000000000001
precision :  0.7649006622516556
recall :  0.7151702786377709
NegPredictiveVal :  0.7325581395348837
speficity :  0.7801857585139319
time:  0.08643782138824463



XGBClassifier
conf_matrix :  [834, 135, 294, 675]
accuracy :  77.86377708978328
fscore :  0.7588532883642496
precision :  0.8333333333333334
recall :  0.6965944272445821
NegPredictiveVal :  0.7393617021276596
speficity :  0.8606811145510835
time:  0.07775692145029704



GradientBoostingClassifier
conf_matrix :  [809, 160, 300, 669]
accuracy :  76.2641898864809
fscore :  0.7441601779755284
precision :  0.8069963811821471
recall :  0.6904024767801857
NegPredictiveVal :  0.7294860234445446
speficity :  0.8348813209494325
time:  0.07386298974355061

In [ ]:
char ngram no stemmer

LogisticRegression
conf_matrix :  [777, 192, 183, 786]
accuracy :  80.6501547987616
fscore :  0.8073959938366718
precision :  0.803680981595092
recall :  0.8111455108359134
NegPredictiveVal :  0.809375
speficity :  0.8018575851393189
time:  0.22742716471354166



SVC
conf_matrix :  [802, 167, 273, 696]
accuracy :  77.29618163054695
fscore :  0.759825327510917
precision :  0.8064889918887601
recall :  0.718266253869969
NegPredictiveVal :  0.746046511627907
speficity :  0.8276573787409701
time:  0.5003648201624553



LinearSVC
conf_matrix :  [739, 230, 192, 777]
accuracy :  78.2249742002064
fscore :  0.7864372469635629
precision :  0.7715988083416088
recall :  0.8018575851393189
NegPredictiveVal :  0.7937701396348013
speficity :  0.762641898864809
time:  0.21390453974405924



NuSVC
conf_matrix :  [795, 174, 245, 724]
accuracy :  78.37977296181631
fscore :  0.7755757900374933
precision :  0.8062360801781737
recall :  0.7471620227038184
NegPredictiveVal :  0.7644230769230769
speficity :  0.8204334365325078
time:  0.4688129822413127



BernoulliNB
conf_matrix :  [563, 406, 136, 833]
accuracy :  72.0330237358101
fscore :  0.7545289855072463
precision :  0.672316384180791
recall :  0.8596491228070176
NegPredictiveVal :  0.8054363376251789
speficity :  0.5810113519091847
time:  0.20873737335205078



MultinomialNB
conf_matrix :  [846, 123, 217, 752]
accuracy :  82.45614035087719
fscore :  0.8156182212581345
precision :  0.8594285714285714
recall :  0.7760577915376677
NegPredictiveVal :  0.7958607714016933
speficity :  0.8730650154798761
time:  0.20457279682159424



ComplementNB
conf_matrix :  [846, 123, 218, 751]
accuracy :  82.4045407636739
fscore :  0.8149755832881171
precision :  0.8592677345537757
recall :  0.7750257997936016
NegPredictiveVal :  0.7951127819548872
speficity :  0.8730650154798761
time:  0.2146298885345459



RandomForestClassifier
conf_matrix :  [780, 189, 242, 727]
accuracy :  77.76057791537667
fscore :  0.7713527851458886
precision :  0.7936681222707423
recall :  0.7502579979360166
NegPredictiveVal :  0.7632093933463796
speficity :  0.804953560371517
time:  0.3569807608922323



KNeighborsClassifier
conf_matrix :  [157, 812, 27, 942]
accuracy :  56.7079463364293
fscore :  0.6918839515240544
precision :  0.5370581527936146
recall :  0.9721362229102167
NegPredictiveVal :  0.8532608695652174
speficity :  0.16202270381836945
time:  0.34412916501363117



DecisionTreeClassifier
conf_matrix :  [694, 275, 274, 695]
accuracy :  71.671826625387
fscore :  0.7168643630737493
precision :  0.7164948453608248
recall :  0.717234262125903
NegPredictiveVal :  0.7169421487603306
speficity :  0.7162022703818369
time:  0.20559974511464438



AdaBoostClassifier
conf_matrix :  [689, 280, 203, 766]
accuracy :  75.07739938080495
fscore :  0.7602977667493795
precision :  0.7323135755258127
recall :  0.7905056759545923
NegPredictiveVal :  0.7724215246636771
speficity :  0.7110423116615067
time:  0.22777922948201498



XGBClassifier
conf_matrix :  [745, 224, 219, 750]
accuracy :  77.14138286893704
fscore :  0.7720020586721564
precision :  0.7700205338809035
recall :  0.7739938080495357
NegPredictiveVal :  0.7728215767634855
speficity :  0.7688338493292054
time:  0.21197362740834555



GradientBoostingClassifier
conf_matrix :  [759, 210, 214, 755]
accuracy :  78.12177502579979
fscore :  0.78076525336091
precision :  0.7823834196891192
recall :  0.7791537667698658
NegPredictiveVal :  0.7800616649537513
speficity :  0.7832817337461301
time:  0.20318210124969482

## stemming results

In [ ]:


# baseline acc

#word
# no stemming, word unigrams
# LogisticRegression
# conf_matrix :  [717, 252, 240, 729]
# accuracy :  74.61300309597523 #new cleaning function -> accuracy :  74.81
# fscore :  0.7476923076923078
# precision :  0.7431192660550459
# recall :  0.7523219814241486
# NegPredictiveVal :  0.7492163009404389
# speficity :  0.739938080495356

#char_wb
#base acc without stemming
# LogisticRegression
# conf_matrix :  [777, 192, 183, 786]
# accuracy :  80.6501547987616
# fscore :  0.8073959938366718
# precision :  0.803680981595092
# recall :  0.8111455108359134
# NegPredictiveVal :  0.809375
# speficity :  0.8018575851393189

corpus_based
tot stemmed words 1298
LogisticRegression
conf_matrix :  [724, 245, 234, 735]
accuracy :  75.28379772961816
fscore :  0.754232939969215
precision :  0.75
recall :  0.7585139318885449
NegPredictiveVal :  0.755741127348643
speficity :  0.7471620227038184

#char_wb
# accuracy :  79.72136222910217
# fscore :  0.7989769820971867

wordCorpus=get_word_corpus(dataSet)[::-1] tried for sv with reversing sorted corpus- accuracy :  74.61300309597523

suffix_removal_stemmer #longest suf from matched suffixes list removed
LogisticRegression
conf_matrix :  [704, 265, 243, 726]
accuracy :  73.78740970072239
fscore :  0.7408163265306122
precision :  0.7325933400605449
recall :  0.7492260061919505
NegPredictiveVal :  0.7434002111932418
speficity :  0.7265221878224974

LogisticRegression #earliast suffix in the matched suffixes list removed
conf_matrix :  [696, 273, 253, 716]
accuracy :  72.85861713106296
fscore :  0.7313585291113381
precision :  0.723963599595551
recall :  0.7389060887512899
NegPredictiveVal :  0.7334035827186512
speficity :  0.718266253869969

















sv 

cutoff val : 2, 'ං','ඃ'..(diacritics) not neglected for both sv in 1.for substring  and 2.sv letter
LogisticRegression
conf_matrix :  [702, 267, 225, 744]
accuracy :  74.61300309597523
fscore :  0.7515151515151515
precision :  0.7359050445103857
recall :  0.7678018575851393
NegPredictiveVal :  0.7572815533980582
speficity :  0.7244582043343654

#char_wb
# accuracy :  80.35


cutoff val :3, 'ං','ඃ'.. not neglected  for sv in 1.for substring 2.sv letter
LogisticRegression
conf_matrix :  [677, 292, 251, 718]
accuracy :  71.9814241486068
fscore :  0.7256189994946943
precision :  0.7108910891089109
recall :  0.7409700722394221
NegPredictiveVal :  0.7295258620689655
speficity :  0.6986584107327142


cutoff val :1, 'ං','ඃ'.. not neglected  for sv in 1.for substring 2.sv letter
LogisticRegression
conf_matrix :  [696, 273, 232, 737]
accuracy :  73.9422084623323
fscore :  0.744820616472966
precision :  0.7297029702970297
recall :  0.760577915376677
NegPredictiveVal :  0.75
speficity :  0.718266253869969

cutoff val :2, 'ං','ඃ'.. neglected for sv in 1.for substring 
LogisticRegression
conf_matrix :  [698, 271, 226, 743]
accuracy :  74.35500515995872
fscore :  0.7493696419566314
precision :  0.7327416173570019
recall :  0.7667698658410733
NegPredictiveVal :  0.7554112554112554
speficity :  0.7203302373581011


cutoff val :3, 'ං','ඃ'.. neglected for sv in 1.for substring 
LogisticRegression
conf_matrix :  [681, 288, 240, 729]
accuracy :  72.75541795665634
fscore :  0.7341389728096677
precision :  0.7168141592920354
recall :  0.7523219814241486
NegPredictiveVal :  0.739413680781759
speficity :  0.7027863777089783

cutoff val :1, 'ං','ඃ'..neglected  for sv in 1.for substring 
LogisticRegression
conf_matrix :  [702, 267, 233, 736]
accuracy :  74.20020639834881
fscore :  0.746450304259635
precision :  0.7337986041874377
recall :  0.759545923632611
NegPredictiveVal :  0.7508021390374332
speficity :  0.7244582043343654

cutoff val :2, 'ං','ඃ'.. neglected for sv in 2.sv letter 
LogisticRegression
conf_matrix :  [682, 287, 236, 733]
accuracy :  73.01341589267287
fscore :  0.7370537958773252
precision :  0.7186274509803922
recall :  0.7564499484004128
NegPredictiveVal :  0.7429193899782135
speficity :  0.7038183694530443

cutoff val :3, 'ං','ඃ'.. neglected for sv in 2.sv letter 
LogisticRegression
conf_matrix :  [666, 303, 238, 731]
accuracy :  72.08462332301342
fscore :  0.7299051422865702
precision :  0.706963249516441
recall :  0.7543859649122807
NegPredictiveVal :  0.7367256637168141
speficity :  0.6873065015479877

cutoff val :1, 'ං','ඃ'.. neglected for sv in 2.sv letter 
LogisticRegression
conf_matrix :  [691, 278, 234, 735]
accuracy :  73.58101135190918
fscore :  0.7416750756811301
precision :  0.7255676209279368
recall :  0.7585139318885449
NegPredictiveVal :  0.7470270270270271
speficity :  0.7131062951496389

cutoff val :2, 'ං','ඃ'.. neglected sv in 1.for substring 2.sv letter
LogisticRegression
conf_matrix :  [677, 292, 235, 734]
accuracy :  72.80701754385966
fscore :  0.7358395989974937
precision :  0.7153996101364523
recall :  0.7574819401444789
NegPredictiveVal :  0.7423245614035088
speficity :  0.6986584107327142

cutoff val :3, 'ං','ඃ'.. neglected sv in 1.for substring 2.sv letter
LogisticRegression
conf_matrix :  [664, 305, 235, 734]
accuracy :  72.13622291021672
fscore :  0.7310756972111554
precision :  0.7064485081809432
recall :  0.7574819401444789
NegPredictiveVal :  0.7385984427141268
speficity :  0.6852425180598555

cutoff val :1, 'ං','ඃ'.. neglected sv in 1.for substring 2.sv letter
LogisticRegression
conf_matrix :  [700, 269, 240, 729]
accuracy :  73.7358101135191
fscore :  0.7412302999491612
precision :  0.7304609218436874
recall :  0.7523219814241486
NegPredictiveVal :  0.7446808510638298
speficity :  0.7223942208462333





sv
max_Entropy
cutoff : ME_Val<=1 and ME_Val>=0.53 #higher the me val higher the accuracy but still less less than normal acc without stemming
conf_matrix :  [715, 254, 245, 724]
accuracy :  74.25180598555211




# **notes**

In [ ]:
stemming can affects important features :  පෙන්නපන්,පෙන්නන්න

for i in d['පෙන්නපං']: print(i)
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}
{'substring': 'පෙන්නප', 'substring_sv_val': 3, 'substring_sv_words': ['පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්']}


for key in d:
  if 'පෙන්න' in key:
    print(key)
    for i in d[key]: print(i)
    print('\n')
පෙන්න
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}


පෙන්නන
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}
{'substring': 'පෙන්නන', 'substring_sv_val': 2, 'substring_sv_words': ['පෙන්නන්න', 'පෙන්නන්නම']}


පෙන්නන්න
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}
{'substring': 'පෙන්නන', 'substring_sv_val': 2, 'substring_sv_words': ['පෙන්නන', 'පෙන්නන්නම']}
{'substring': 'පෙන්නන්න', 'substring_sv_val': 1, 'substring_sv_words': ['පෙන්නන්නම']}


පෙන්නන්නම
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}
{'substring': 'පෙන්නන', 'substring_sv_val': 2, 'substring_sv_words': ['පෙන්නන', 'පෙන්නන්න']}
{'substring': 'පෙන්නන්න', 'substring_sv_val': 1, 'substring_sv_words': ['පෙන්නන්න']}


පෙන්නපං
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}
{'substring': 'පෙන්නප', 'substring_sv_val': 3, 'substring_sv_words': ['පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්']}


පෙන්නපන්
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}
{'substring': 'පෙන්නප', 'substring_sv_val': 3, 'substring_sv_words': ['පෙන්නපං', 'පෙන්නපන්කෝ', 'පෙන්නපාන්']}
{'substring': 'පෙන්නපන', 'substring_sv_val': 1, 'substring_sv_words': ['පෙන්නපන්කෝ']}


පෙන්නපන්කෝ
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}
{'substring': 'පෙන්නප', 'substring_sv_val': 3, 'substring_sv_words': ['පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපාන්']}
{'substring': 'පෙන්නපන', 'substring_sv_val': 1, 'substring_sv_words': ['පෙන්නපන්']}


පෙන්නපාන්
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}
{'substring': 'පෙන්නප', 'substring_sv_val': 3, 'substring_sv_words': ['පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ']}


පෙන්නයි
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}


පෙන්නලා
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}


පෙන්නවූ
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}


පෙන්නහන්
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}


පෙන්නාවි
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}


පෙන්නුවා
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්නේ']}
{'substring': 'පෙන්නුව', 'substring_sv_val': 1, 'substring_sv_words': ['පෙන්නුවේ']}


පෙන්නුවේ
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නෙ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නෙ', 'පෙන්නේ']}
{'substring': 'පෙන්නුව', 'substring_sv_val': 1, 'substring_sv_words': ['පෙන්නුවා']}


පෙන්නෙ
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නේ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නේ']}


පෙන්නේ
{'substring': 'ප', 'substring_sv_val': 570, 'substring_sv_words': ['පංචන්', 'පඋ', 'පක', 'පකක්', 'පකඩ', 'පකය', 'පකයන්ගෙ', 'පකයල', 'පකයා', 'පකයි', 'පකයියෝ', 'පකයිල', 'පක්කෝ', 'පක්ෂ', 'පක්ෂෙට', 'පකිර්නෙ', 'පකිස්ටන්', 'පකිස්තානේ', 'පකිස්බූරුවොයි', 'පකිස්\u200dතානෙ', 'පකීර්ලට', 'පකේ', 'පකො', 'පකෝ', 'පගර', 'පචනෙ', 'පචමයි', 'පටන්', 'පටන්ගතයුතු', 'පටන්ගත්තා', 'පටන්ගත්තේ', 'පටන්ගන්න', 'පටලවලා', 'පට්ට', 'පට්ටපල්', 'පට්ටම', 'පට්ටම්', 'පට්ටයි', 'පට්ට්ට්ට', 'පට්ටෙට', 'පටිට', 'පඩ', 'පඩයක්', 'පඩි', 'පඩික්කම්', 'පණ', 'පතන', 'පතනව', 'පතනවා', 'පතමි', 'පතමු', 'පත්', 'පත්කරගැනීමට', 'පත්කරන්න', 'පත්කරපු', 'පත්තරකාරයෙක්', 'පත්වුයේ', 'පත්වෙලා', 'පතුරවන්නයි', 'පද', 'පදක්කම', 'පදක්කමක්', 'පදක්කම්', 'පදක්කම්ලාබී', 'පදින්ඩ', 'පන', 'පනහක්', 'පනහෙ', 'පන්', 'පන්ඩිත', 'පන්තියේ', 'පන්දම්', 'පන්දු', 'පන්දුයවන්නෙක්', 'පන්දුව', 'පන්දුවට', 'පන්දුවෙන්', 'පන්නන්න', 'පන්නල', 'පනා', 'පනින', 'පනින්න', 'පනින්නෙ', 'පනුවො', 'පනුවෝ', 'පපරේ', 'පපුලුවන්', 'පපුවක්', 'පපුවට', 'පමණක්', 'පමණකි', 'පම්පොරි', 'පම්පෝරි', 'පඹයොන්ට', 'පඹයෝ', 'පයින', 'පයින්', 'පයිසර්', 'පයිසර්ගෙ', 'පර', 'පරක්කු', 'පරඩැල්', 'පරණ', 'පරද්දන', 'පරද්දනවා', 'පරද්දන්න', 'පරදිද්දි', 'පරදින', 'පරදිනකොට', 'පරදිනවා', 'පරදින්නයි', 'පරදින්නෙ', 'පරන', 'පරපුරක්', 'පරපුරට', 'පරපුරේ', 'පරම්පරාවක්', 'පරම්පරාවට', 'පරම්පරාවේ', 'පරයන්ට', 'පරස්න', 'පර්නැන්ඩු', 'පර්සි', 'පරාජ', 'පරාජය', 'පරාජයන්', 'පරාජයේ', 'පරාද', 'පරාදද', 'පරාදයි', 'පරාධයේ', 'පරිදි', 'පරිනත', 'පරිනතභාවයත්', 'පරිපාලකයෝ', 'පරිපාලනය', 'පරිපාලනේ', 'පරිසම්', 'පරිසමාප්ත', 'පරිස්සමෙන්', 'පරෙ', 'පල', 'පලක', 'පලගේ', 'පලදෙනවා', 'පලමු', 'පලයං', 'පලයන්', 'පලයන්බං', 'පලයව්', 'පලවෙනි', 'පල්', 'පල්ලෙහාට', 'පලි', 'පලිය', 'පලියට', 'පලු', 'පලෙන්න', 'පව', 'පව්', 'පව්කරොත්', 'පව්ල', 'පව්ලේ', 'පවිච්කරා', 'පවිත්\u200dරා', 'පවු', 'පවුලකින්', 'පවුල්', 'පවුලේ', 'පවුුල්', 'පස්ස', 'පස්සට', 'පස්සෙ', 'පස්සේ', 'පස්සේ', 'පසු', 'පසුගියකාලේ', 'පසුතැවීමෙන්', 'පසුව', 'පහකට', 'පහක්', 'පහක්වත්', 'පහත', 'පහත්', 'පහර', 'පහල', 'පහලට', 'පහසු', 'පහසුකම්', 'පහළ', 'පහාරදෙන', 'පහු', 'පහුගිය', 'පළකරන්න', 'පළලා', 'පළවෙනි', 'ප්රේක්ෂකයන්ට', 'ප්ලයන්', 'ප්ලය්', 'ප්ලයෙර්', 'ප්ල්ය්\u200dර', 'ප්ලේ', 'ප්ලේයරය', 'ප්ලේයරස්ලා', 'ප්ලොට්ස්', 'ප්\u200dරකෝටි', 'ප්\u200dරගතිය', 'ප්\u200dරණාමයක්', 'ප්\u200dරතම', 'ප්\u200dරතික්ෂේප', 'ප්\u200dරතිචාරද', 'ප්\u200dරතිපලය', 'ප්\u200dරතිඵල', 'ප්\u200dරතිලාභයක්', 'ප්\u200dරතිවාදී', 'ප්\u200dරදිප්ල', 'ප්\u200dරධාන', 'ප්\u200dරධානියා', 'ප්\u200dරනාන්දු', 'ප්\u200dරනාන්දුට', 'ප්\u200dරනාමය', 'ප්\u200dරභාකර්', 'ප්\u200dරමාණය', 'ප්\u200dරරස්න', 'ප්\u200dරවේශම්වෙයන්', 'ප්\u200dරශන්ශා', 'ප්\u200dරශ්ණ', 'ප්\u200dරශ්ණයක්', 'ප්\u200dරශ්න', 'ප්\u200dරශ්නය', 'ප්\u200dරශ්නයක්', 'ප්\u200dරශ්නේ', 'ප්\u200dරසන්න', 'ප්\u200dරසන්නටත්', 'ප්\u200dරසන්නව', 'ප්\u200dරසිඩ්ඩ', 'ප්\u200dරසිද්ද', 'ප්\u200dරසිද්දියෙ', 'ප්\u200dරසිද්දියේ', 'ප්\u200dරසිද්ධ', 'ප්\u200dරසිද්ධියේම', 'ප්\u200dරහාරයේ', 'ප්\u200dරාදේශීය', 'ප්\u200dරාර්ථනා', 'ප්\u200dරෙක්ෂකයයෝ', 'ප්\u200dරෙක්ෂකයො', 'ප්\u200dරෙහාරය', 'ප්\u200dරේක්ශකයෙක්', 'ප්\u200dරේක්ශකයෝ', 'ප්\u200dරේක්ෂකයන්', 'ප්\u200dරේක්ෂකයන්ට', 'ප්\u200dරේශ්ක්ශකයෝ', 'පාකේ', 'පාට', 'පාටක්', 'පාඩම්', 'පාඩු', 'පාඩුවක්', 'පාඩුවේ', 'පාත්', 'පාන්', 'පාන්කඩයෝ', 'පාර', 'පාරක්', 'පාරද', 'පාරදිගේ', 'පාරමී', 'පාරවල්', 'පාර්ථනා', 'පාර්ලිමේන්තු', 'පාර්ලිමේන්තුවට', 'පාර්ලිමේන්තුවෙ', 'පාර්ලිමේන්තුවේ', 'පාරාජය', 'පාරාදද', 'පාරෙ', 'පාරේ', 'පාල', 'පාලක', 'පාලකයන්', 'පාලකයන්ගෙයි', 'පාලකයිනි', 'පාලකයො', 'පාලකයෝ', 'පාලට', 'පාලන', 'පාලනය', 'පාලනයට', 'පාලනේ', 'පාලුවන්ට', 'පාවලාදෙනවා', 'පාවා', 'පාවිච්චි', 'පාසල්', 'පාසලේ', 'පාසැලෙන්', 'පාහර', 'පාළනයක්', 'පාුර්තනා', 'පැකට්', 'පැකටි', 'පැටව්', 'පැටවු', 'පැටියෙක්', 'පැණි', 'පැත්ත', 'පැත්තක', 'පැත්තකට', 'පැත්තකින්', 'පැත්තට', 'පැත්තෙ', 'පැත්තෙන්', 'පැත්තේ', 'පැතුම්', 'පැදුනාම', 'පැනපියවු', 'පැනලා', 'පැරදිල', 'පැරදිලා', 'පැරදුණත්', 'පැරදුණාට', 'පැරදුණුත්', 'පැරදුණොත්', 'පැරදුනත්', 'පැරදුනම', 'පැරදුනා', 'පැරදුනාට', 'පැරදුනානමි', 'පැරදුනාම', 'පැරදුනොත්', 'පැරදුනේ', 'පැරදුනේම', 'පැරදුම', 'පැරදෙනවා', 'පැලෙන්න', 'පැවසුවේය', 'පැෂතුමි', 'පැහැදිලි', 'පැහැදිලිව', 'පැහැදිලිවම', 'පිචචෙන', 'පිච්', 'පිච්චෙන', 'පිට', 'පිටම', 'පිටයි', 'පිටරටවලට', 'පිට්ටනියේ', 'පිට්ටුවෝ', 'පිටින්', 'පිටිය', 'පිටියේ', 'පිත්ත', 'පිත්තක්', 'පිත්තෙන්', 'පිත්තෙන්න්', 'පිති', 'පිතිකරැට', 'පිතිකරැවන්ට', 'පිතිකරු', 'පිතිකරුව', 'පිතිකරුවන්', 'පිතිකරුවා', 'පිතිකරුවෙක්', 'පිතිකරුවො', 'පිනකට', 'පිනක්', 'පිනට', 'පින්', 'පින්තුරය', 'පින්තූරයක්', 'පින්නයෝ', 'පිනිසය', 'පියතුමන්', 'පිරි', 'පිරිපුන්', 'පිරිමහන', 'පිරිමියෙක්', 'පිරිස', 'පිරිසක්', 'පිරුනු', 'පිරෙනව', 'පිරෙන්න', 'පිලක්', 'පිලි', 'පිලිගත්', 'පිලිගත්තු', 'පිලිගන්න', 'පිලිතුරු', 'පිලිඹඳ', 'පිලිවෙලට', 'පිස්සන්ට', 'පිස්සු', 'පිස්සුද', 'පිස්සුව', 'පිස්සෙක්', 'පිස්සොන්ගෙ', 'පිහා', 'පිහාට', 'පිහිට', 'පිහිටයයි', 'පිහිටයි', 'පිහිටෙන්', 'පිළට', 'පිළිගන්නෙ', 'පිළිම', 'පිළිමයක්', 'පිළිවෙළක්', 'පීඩනය', 'පීඩනයක්', 'පුංගක්කාර', 'පුංගක්කාරයෝ', 'පුංචි', 'පුක', 'පුකට', 'පුකයි', 'පුකෙන්', 'පුකේ', 'පුජාන්තතය', 'පුජිත', 'පුට', 'පුටුවක', 'පුත', 'පුතනුවකු', 'පුතල', 'පුතලට', 'පුතා', 'පුතාට', 'පුතාල', 'පුතාලනේ', 'පුතෙක්', 'පුතේ', 'පුතො', 'පුද', 'පුදන', 'පුද්ගල', 'පුද්ගලයන්', 'පුද්ගලයන්ගේ', 'පුද්ගලයන්ට', 'පුද්ගලයා', 'පුද්ගලයෙක්', 'පුද්ගලික', 'පුද්ගලිකව', 'පුදුම', 'පුදුමයක්', 'පුදූම', 'පුන්චි', 'පුප', 'පුම්බලා', 'පුරවන්න', 'පුරවැසියකු', 'පුරවැසියෙක්', 'පුර්ථානා', 'පුරා', 'පුරාම', 'පුරාවෘත්තය', 'පුරැද්දට', 'පුරුදු', 'පුරෝලා', 'පුලස්ති', 'පුලුවන්', 'පුලුවන්නම්', 'පුලුවන්නේ', 'පුවතට', 'පුශ්න', 'පුහුණු', 'පුහුණුකරු', 'පුහුණුකරුගේ', 'පුහුණුකරුවෙක්', 'පුහුණුකරුවෙකුගෙන්', 'පුහුණුව', 'පුහුනු', 'පුහුනුවීම්', 'පුළුවන්', 'පූජාසනයේ', 'පූජිත', 'පූජීත', 'පූරූදූ', 'පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි', 'පෙපරමිනිටි', 'පෙරදුනට', 'පෙරහැර', 'පෙරහැරක්', 'පෙරහැරට', 'පෙරහැරේම', 'පෙර්රේතයා', 'පෙරාගෙන', 'පෙරේතයි', 'පෙරේදා', 'පෙරේරා', 'පෙල', 'පෙලට', 'පෙලන්න', 'පෙසන්නල', 'පොඩි', 'පේන', 'පේනවද', 'පේනවනේ', 'පේනවා', 'පේන්න', 'පේන්නේ', 'පොට්ට', 'පොඩ්', 'පොඩ්ඩක්', 'පොඩි', 'පොඩිඩක්', 'පොඩිම', 'පොඩී', 'පොදියේ', 'පොදු', 'පොදුයි', 'පොදුවේ', 'පොන්ටින්', 'පොන්න', 'පොන්නය', 'පොන්නයගේ', 'පොන්නයා', 'පොන්නයෙක්', 'පොන්නයෙක්ද', 'පොන්නයො', 'පොන්නයෝ', 'පොන්නා', 'පොන්සියො', 'පොප්සියව', 'පොර', 'පොරක්', 'පොරක්ලු', 'පොරටෝක්', 'පොරවල්', 'පොරවෙන්න', 'පොලව', 'පොලවට', 'පොල්', 'පොලිම්', 'පොලිසිය', 'පොලිසියට', 'පොලු', 'පොලොව', 'පොවන්න', 'පොවල', 'පොළව', 'පෝන්', 'පෝලිමේ', 'පෝස්ට්', 'පෞ', 'පෞද්ගලික']}
{'substring': 'පෙන', 'substring_sv_val': 25, 'substring_sv_words': ['පෙනවනේ', 'පෙන්ඩ', 'පෙන්දන්ට', 'පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ', 'පෙන්වන්න', 'පෙන්වල', 'පෙනි', 'පෙනෙ', 'පෙනෙනවා', 'පෙනෙයි']}
{'substring': 'පෙන්න', 'substring_sv_val': 16, 'substring_sv_words': ['පෙන්න', 'පෙන්නන', 'පෙන්නන්න', 'පෙන්නන්නම', 'පෙන්නපං', 'පෙන්නපන්', 'පෙන්නපන්කෝ', 'පෙන්නපාන්', 'පෙන්නයි', 'පෙන්නලා', 'පෙන්නවූ', 'පෙන්නහන්', 'පෙන්නාවි', 'පෙන්නුවා', 'පෙන්නුවේ', 'පෙන්නෙ']}
